5.2.2. 순전파 코드 구현

In [3]:
import numpy as np
import matplotlib.pyplot as plt

# -- 각 설정값 --
n_time = 10      # 시점 수
n_in = 1         # 입력층 뉴런 수
n_mid = 20       # 은닉층 뉴런 수
n_out = 1        # 출력층 뉴런 수

eta = 0.01       # 학습률
epochs = 101
batch_size = 8
interval = 10    # 경과 표시 간격

def sigmoid(x):
    return 1/(1+np.exp(-x))

# -- 훈련 데이터 생성 --
sin_x = np.linspace(-2*np.pi, 2*np.pi)
print(len(sin_x))
# print(sin_x)
# plt.plot(range(len(sin_x)), sin_x)
# plt.show()

sin_y = np.sin(sin_x) + 0.1*np.random.randn(len(sin_x))
# plt.plot(range(len(sin_y)), sin_y)
# plt.show()

n_sample = len(sin_x) - n_time    # 샘플 수
input_data = np.zeros((n_sample, n_time, n_in))   # 입력
correct_data = np.zeros((n_sample, n_out))        # 정답

for i in range(0, n_sample):
    print(i)
    input_data[i] = sin_y[i:i+n_time].reshape(-1,1)
    # 정답은 입력보다 한 시점 후
    print(input_data[i])

50
0
[[-0.04575652]
 [ 0.32131565]
 [ 0.57956992]
 [ 0.54722715]
 [ 0.88952586]
 [ 1.03368589]
 [ 1.13428403]
 [ 1.03461081]
 [ 1.04568075]
 [ 0.75218414]]
1
[[0.32131565]
 [0.57956992]
 [0.54722715]
 [0.88952586]
 [1.03368589]
 [1.13428403]
 [1.03461081]
 [1.04568075]
 [0.75218414]
 [0.41043847]]
2
[[0.57956992]
 [0.54722715]
 [0.88952586]
 [1.03368589]
 [1.13428403]
 [1.03461081]
 [1.04568075]
 [0.75218414]
 [0.41043847]
 [0.38938836]]
3
[[ 0.54722715]
 [ 0.88952586]
 [ 1.03368589]
 [ 1.13428403]
 [ 1.03461081]
 [ 1.04568075]
 [ 0.75218414]
 [ 0.41043847]
 [ 0.38938836]
 [-0.00669395]]
4
[[ 0.88952586]
 [ 1.03368589]
 [ 1.13428403]
 [ 1.03461081]
 [ 1.04568075]
 [ 0.75218414]
 [ 0.41043847]
 [ 0.38938836]
 [-0.00669395]
 [-0.12205961]]
5
[[ 1.03368589]
 [ 1.13428403]
 [ 1.03461081]
 [ 1.04568075]
 [ 0.75218414]
 [ 0.41043847]
 [ 0.38938836]
 [-0.00669395]
 [-0.12205961]
 [-0.46305989]]
6
[[ 1.13428403]
 [ 1.03461081]
 [ 1.04568075]
 [ 0.75218414]
 [ 0.41043847]
 [ 0.38938836]
 [-0.00